# Day13_0: 회귀 모델 (Regression Models)

## 📚 학습 목표

**Part 1: 기초 - 선형 모델**
1. 선형 회귀(Linear Regression)의 원리와 구현 이해하기
2. Ridge Regression (L2 정규화) 활용법 익히기
3. Lasso Regression (L1 정규화)로 특성 선택하기
4. ElasticNet으로 L1+L2 조합 활용하기
5. 회귀 평가 지표(MSE, RMSE, MAE, R2) 이해하기

**Part 2: 심화 - 비선형 모델 & 튜닝**
1. Decision Tree Regressor로 비선형 패턴 학습하기
2. Random Forest Regressor로 앙상블 회귀 구현하기
3. XGBoost/LightGBM으로 고성능 회귀 달성하기
4. GridSearchCV/RandomizedSearchCV로 하이퍼파라미터 튜닝하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 선형 회귀 | 기본 예측, 해석 가능한 모델 | 매출 예측, 가격 산정 |
| Ridge/Lasso | 과적합 방지, 특성 선택 | 다중공선성 처리, 핵심 변수 추출 |
| 앙상블 모델 | 고성능 예측 | 주택 가격, 수요 예측 |
| 하이퍼파라미터 튜닝 | 모델 최적화 | Kaggle 경진대회, 실무 배포 |

**분석가 관점**: 회귀 분석은 데이터 분석의 핵심입니다. 부동산 가격 예측, 매출 예측, 수요 예측 등 비즈니스의 거의 모든 예측 문제가 회귀로 시작됩니다!

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn 모델
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# sklearn 유틸리티
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Gradient Boosting 라이브러리
try:
    import xgboost as xgb
    print("XGBoost 버전:", xgb.__version__)
except ImportError:
    print("XGBoost 설치 필요: pip install xgboost")

try:
    import lightgbm as lgb
    print("LightGBM 버전:", lgb.__version__)
except ImportError:
    print("LightGBM 설치 필요: pip install lightgbm")

try:
    import catboost as cb
    print("CatBoost 버전:", cb.__version__)
except ImportError:
    print("CatBoost 설치 필요: pip install catboost")

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
np.random.seed(42)

print("환경 설정 완료!")

: 

## 데이터셋 준비: California Housing

캘리포니아 주택 가격 데이터를 사용합니다. (Boston Housing은 윤리적 문제로 sklearn에서 제거됨)

In [ ]:
from sklearn.datasets import fetch_california_housing

# 데이터 로드
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target  # 중간 주택 가격 (단위: $100,000)

print("데이터 크기:", df.shape)
print("\n컬럼 설명:")
print("- MedInc: 중간 소득")
print("- HouseAge: 주택 연식")
print("- AveRooms: 평균 방 개수")
print("- AveBedrms: 평균 침실 개수")
print("- Population: 지역 인구")
print("- AveOccup: 평균 거주자 수")
print("- Latitude: 위도")
print("- Longitude: 경도")
print("- MedHouseVal: 중간 주택 가격 (타겟, $100,000 단위)")

df.head()

In [ ]:
# 기본 통계
df.describe()

In [ ]:
# 타겟 변수 분포 확인
fig = px.histogram(df, x='MedHouseVal', nbins=50, 
                   title='주택 가격 분포 (단위: $100,000)',
                   labels={'MedHouseVal': '중간 주택 가격'})
fig.show()

In [ ]:
# 학습/테스트 데이터 분리
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"학습 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")

In [ ]:
# 스케일링 (선형 모델용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료!")

---

# Part 1: 기초 - 선형 모델

---

## 1.1 선형 회귀 (Linear Regression)

선형 회귀는 가장 기본적인 회귀 모델로, 특성과 타겟 간의 선형 관계를 학습합니다.

$$y = w_1x_1 + w_2x_2 + ... + w_nx_n + b$$

**장점**: 해석 가능, 빠른 학습  
**단점**: 비선형 관계 학습 불가, 이상치에 민감

In [ ]:
# 선형 회귀 모델 학습
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# 예측
y_pred_lr = lr_model.predict(X_test_scaled)

# 성능 평가
print("=== 선형 회귀 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_lr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_lr):.4f}")

In [ ]:
# 회귀 계수 확인 (특성 중요도)
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

fig = px.bar(coef_df, x='Coefficient', y='Feature', orientation='h',
             title='선형 회귀 계수 (특성 영향도)',
             color='Coefficient', color_continuous_scale='RdBu')
fig.show()

### 💡 실무 예시: 회귀 계수 해석

- **양수 계수**: 특성이 증가하면 주택 가격도 증가
- **음수 계수**: 특성이 증가하면 주택 가격 감소
- **절대값 크기**: 영향력의 크기 (스케일링 후 비교 가능)

---

## 1.2 Ridge Regression (L2 정규화)

Ridge는 계수의 크기에 페널티를 부여하여 과적합을 방지합니다.

$$Loss = MSE + \alpha \sum w_i^2$$

**alpha**: 정규화 강도 (클수록 계수가 작아짐)

In [ ]:
# Ridge 모델 학습
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)

# 예측
y_pred_ridge = ridge_model.predict(X_test_scaled)

print("=== Ridge Regression 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_ridge):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_ridge)):.4f}")

In [ ]:
# alpha 값에 따른 계수 변화 시각화
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
coef_results = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    for i, col in enumerate(X.columns):
        coef_results.append({
            'alpha': alpha,
            'Feature': col,
            'Coefficient': ridge.coef_[i]
        })

coef_df = pd.DataFrame(coef_results)

fig = px.line(coef_df, x='alpha', y='Coefficient', color='Feature',
              title='Ridge: Alpha에 따른 계수 변화',
              log_x=True)
fig.show()

### 💡 실무 예시: Ridge 활용 시점

- **다중공선성 문제**: 특성 간 상관관계가 높을 때
- **과적합 방지**: 학습 데이터에 과하게 맞출 때
- **안정적인 예측**: 모든 특성을 유지하면서 계수만 축소

---

## 1.3 Lasso Regression (L1 정규화)

Lasso는 계수를 0으로 만들 수 있어 **자동 특성 선택** 효과가 있습니다.

$$Loss = MSE + \alpha \sum |w_i|$$

In [ ]:
# Lasso 모델 학습
lasso_model = Lasso(alpha=0.01)
lasso_model.fit(X_train_scaled, y_train)

# 예측
y_pred_lasso = lasso_model.predict(X_test_scaled)

print("=== Lasso Regression 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_lasso):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lasso)):.4f}")

# 0이 아닌 계수 확인
non_zero = np.sum(lasso_model.coef_ != 0)
print(f"\n선택된 특성 수: {non_zero}/{len(X.columns)}")

In [ ]:
# Lasso 계수 시각화
lasso_coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

fig = px.bar(lasso_coef_df, x='Coefficient', y='Feature', orientation='h',
             title='Lasso 계수 (0인 특성 = 선택되지 않음)',
             color='Coefficient', color_continuous_scale='RdBu')
fig.show()

### 💡 실무 예시: Lasso로 핵심 변수 추출

- 100개 특성 중 핵심 10개만 선택
- 불필요한 변수 자동 제거
- 모델 해석 용이성 증가

---

## 1.4 ElasticNet (L1 + L2)

Ridge와 Lasso의 장점을 결합합니다.

$$Loss = MSE + \alpha \cdot l1\_ratio \cdot \sum|w_i| + \alpha \cdot (1-l1\_ratio) \cdot \sum w_i^2$$

In [ ]:
# ElasticNet 모델 학습
elastic_model = ElasticNet(alpha=0.01, l1_ratio=0.5)  # l1_ratio: L1 비율
elastic_model.fit(X_train_scaled, y_train)

# 예측
y_pred_elastic = elastic_model.predict(X_test_scaled)

print("=== ElasticNet 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_elastic):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_elastic)):.4f}")

---

## 1.5 회귀 평가 지표 정리

| 지표 | 수식 | 해석 |
|------|------|------|
| MSE | $\frac{1}{n}\sum(y_i - \hat{y}_i)^2$ | 평균 제곱 오차 (이상치에 민감) |
| RMSE | $\sqrt{MSE}$ | 원래 단위로 해석 가능 |
| MAE | $\frac{1}{n}\sum|y_i - \hat{y}_i|$ | 평균 절대 오차 (이상치에 강건) |
| R2 | $1 - \frac{SS_{res}}{SS_{tot}}$ | 설명력 (0~1, 높을수록 좋음) |

In [ ]:
# 평가 지표 함수 정의
def evaluate_model(y_true, y_pred, model_name="Model"):
    """회귀 모델 평가 지표 계산"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }

# 선형 모델 비교
linear_results = [
    evaluate_model(y_test, y_pred_lr, "Linear Regression"),
    evaluate_model(y_test, y_pred_ridge, "Ridge"),
    evaluate_model(y_test, y_pred_lasso, "Lasso"),
    evaluate_model(y_test, y_pred_elastic, "ElasticNet")
]

linear_df = pd.DataFrame(linear_results)
linear_df

---

# Part 2: 심화 - 비선형 모델 & 튜닝

---

## 2.1 Decision Tree Regressor

트리 기반 모델은 비선형 관계를 학습할 수 있습니다.

In [ ]:
# Decision Tree 모델 (스케일링 불필요)
dt_model = DecisionTreeRegressor(max_depth=10, random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("=== Decision Tree 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_dt):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_dt)):.4f}")

In [ ]:
# 특성 중요도 시각화
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False)

fig = px.bar(importance_df, x='Importance', y='Feature', orientation='h',
             title='Decision Tree 특성 중요도')
fig.show()

---

## 2.2 Random Forest Regressor

여러 트리의 앙상블로 더 안정적인 예측을 제공합니다.

In [ ]:
# Random Forest 모델
rf_model = RandomForestRegressor(
    n_estimators=100,  # 트리 개수
    max_depth=15,
    random_state=42,
    n_jobs=-1  # 병렬 처리
)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("=== Random Forest 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_rf):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.4f}")

In [ ]:
# Random Forest 특성 중요도
rf_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

fig = px.bar(rf_importance_df, x='Importance', y='Feature', orientation='h',
             title='Random Forest 특성 중요도')
fig.show()

### 💡 실무 예시: Random Forest 장점

- **과적합 방지**: 여러 트리의 평균으로 분산 감소
- **특성 선택 불필요**: 자동으로 중요 특성 학습
- **이상치에 강건**: 트리 기반 모델 특성

---

## 2.3 Gradient Boosting (XGBoost, LightGBM)

순차적으로 트리를 학습하며 이전 모델의 오차를 보완합니다.

In [ ]:
# XGBoost 모델
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbosity=0
)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("=== XGBoost 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_xgb):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_xgb)):.4f}")

In [ ]:
# LightGBM 모델
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train, y_train)

y_pred_lgb = lgb_model.predict(X_test)

print("=== LightGBM 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_lgb):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lgb)):.4f}")

### 💡 XGBoost vs LightGBM 비교

| 특성 | XGBoost | LightGBM |
|------|---------|----------|
| 트리 성장 | Level-wise | Leaf-wise |
| 속도 | 빠름 | 더 빠름 |
| 메모리 | 보통 | 효율적 |
| 대용량 | 적합 | 매우 적합 |
| 정확도 | 높음 | 높음 |

---

## 2.4 CatBoost (Categorical Boosting)

**CatBoost**는 Yandex에서 개발한 Gradient Boosting 라이브러리로, **범주형 특성을 자동으로 처리**하는 것이 특징입니다.

### CatBoost의 핵심 특징

1. **Ordered Target Statistics**: 범주형 변수 인코딩 시 타겟 누수(Target Leakage) 방지
2. **범주형 자동 처리**: 원-핫 인코딩 없이 `cat_features` 지정만으로 처리
3. **대칭 트리(Oblivious Trees)**: 모든 노드가 동일 조건으로 분할 → 빠른 예측 속도
4. **과적합 방지**: 기본적으로 강력한 정규화 내장

In [ ]:
# CatBoost 모델
cat_model = cb.CatBoostRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    verbose=0  # 출력 숨김
)
cat_model.fit(X_train, y_train)

y_pred_cat = cat_model.predict(X_test)

print("=== CatBoost 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_cat):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_cat)):.4f}")

### 💡 부스팅 3대장 비교: XGBoost vs LightGBM vs CatBoost

| 특성 | XGBoost | LightGBM | CatBoost |
|------|---------|----------|----------|
| **개발사** | DMLC | Microsoft | Yandex |
| **트리 성장** | Level-wise | Leaf-wise | Symmetric (Oblivious) |
| **범주형 처리** | 직접 인코딩 필요 | 직접 인코딩 필요 | **자동 처리** |
| **속도** | 빠름 | **가장 빠름** | 보통 |
| **과적합 방지** | 정규화 파라미터 | 정규화 파라미터 | **기본 내장** |
| **GPU 지원** | ✅ | ✅ | ✅ |
| **핵심 장점** | 안정성, 범용성 | 속도, 메모리 효율 | 범주형 자동 처리 |
| **추천 상황** | 일반적 선택 | 대용량 데이터 | 범주형 특성 많을 때 |

> **실무 팁**: 
> - 범주형 특성이 많다면 → **CatBoost** 우선 시도
> - 데이터가 매우 크다면 → **LightGBM** 추천
> - 안정적인 성능이 필요하다면 → **XGBoost** 선택

---

## 2.5 하이퍼파라미터 튜닝

### GridSearchCV: 모든 조합 탐색

In [ ]:
# GridSearchCV 예제 (Random Forest)
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(
    rf_base,
    param_grid,
    cv=3,  # 3-fold CV
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("\n최적 파라미터:", grid_search.best_params_)
print(f"최적 CV RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

In [ ]:
# GridSearch 결과 시각화
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results['RMSE'] = np.sqrt(-cv_results['mean_test_score'])

fig = px.scatter(cv_results, 
                 x='param_max_depth', 
                 y='RMSE',
                 color='param_n_estimators',
                 size='param_min_samples_split',
                 title='GridSearchCV 결과: 파라미터별 RMSE',
                 labels={'param_max_depth': 'max_depth', 'param_n_estimators': 'n_estimators'})
fig.show()

### RandomizedSearchCV: 무작위 탐색 (대규모 탐색에 효율적)

In [ ]:
from scipy.stats import randint, uniform

# RandomizedSearchCV (XGBoost)
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4)
}

xgb_base = xgb.XGBRegressor(random_state=42, verbosity=0)

random_search = RandomizedSearchCV(
    xgb_base,
    param_dist,
    n_iter=20,  # 탐색 횟수
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("\n최적 파라미터:", random_search.best_params_)
print(f"최적 CV RMSE: {np.sqrt(-random_search.best_score_):.4f}")

In [ ]:
# 최적 모델로 테스트 예측
best_xgb = random_search.best_estimator_
y_pred_best = best_xgb.predict(X_test)

print("=== 튜닝된 XGBoost 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_best):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_best)):.4f}")

---

## 2.6 모델 비교 종합

In [ ]:
# 모든 모델 결과 종합
all_results = [
    evaluate_model(y_test, y_pred_lr, "Linear Regression"),
    evaluate_model(y_test, y_pred_ridge, "Ridge"),
    evaluate_model(y_test, y_pred_lasso, "Lasso"),
    evaluate_model(y_test, y_pred_elastic, "ElasticNet"),
    evaluate_model(y_test, y_pred_dt, "Decision Tree"),
    evaluate_model(y_test, y_pred_rf, "Random Forest"),
    evaluate_model(y_test, y_pred_xgb, "XGBoost"),
    evaluate_model(y_test, y_pred_lgb, "LightGBM"),
    evaluate_model(y_test, y_pred_cat, "CatBoost"),
    evaluate_model(y_test, y_pred_best, "XGBoost (Tuned)")
]

results_df = pd.DataFrame(all_results).sort_values('RMSE')
results_df

In [ ]:
# 모델 비교 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['RMSE (낮을수록 좋음)', 'R2 Score (높을수록 좋음)'])

# RMSE 바 차트
fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['RMSE'], name='RMSE',
           marker_color='indianred'),
    row=1, col=1
)

# R2 바 차트
fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['R2'], name='R2',
           marker_color='seagreen'),
    row=1, col=2
)

fig.update_layout(title='회귀 모델 성능 비교', height=500, showlegend=False)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# 예측 vs 실제 산점도 (최고 성능 모델)
best_model_name = results_df.iloc[0]['Model']

# 예측값 선택
pred_map = {
    'Linear Regression': y_pred_lr,
    'Ridge': y_pred_ridge,
    'Lasso': y_pred_lasso,
    'ElasticNet': y_pred_elastic,
    'Decision Tree': y_pred_dt,
    'Random Forest': y_pred_rf,
    'XGBoost': y_pred_xgb,
    'LightGBM': y_pred_lgb,
    'CatBoost': y_pred_cat,
    'XGBoost (Tuned)': y_pred_best
}

best_pred = pred_map[best_model_name]

fig = px.scatter(x=y_test, y=best_pred, opacity=0.3,
                 title=f'{best_model_name}: 실제 vs 예측',
                 labels={'x': '실제 가격', 'y': '예측 가격'})

# 대각선 추가 (완벽한 예측 선)
fig.add_trace(go.Scatter(x=[0, 5], y=[0, 5], mode='lines',
                         line=dict(color='red', dash='dash'),
                         name='Perfect'))
fig.show()

### 💡 실무 팁: 모델 선택 기준

1. **성능 우선**: RMSE/R2가 가장 중요하면 XGBoost/LightGBM
2. **해석 필요**: 비즈니스 이해 필요하면 선형 회귀
3. **속도 중요**: 실시간 예측이면 간단한 모델
4. **균형점**: Random Forest가 좋은 선택

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 선형 회귀 기본 ⭐

**문제**: LinearRegression 모델을 학습하고 R2 스코어를 계산하세요.

```python
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# X_train, X_test, y_train, y_test는 이미 준비됨
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. Ridge 정규화 ⭐

**문제**: alpha=10인 Ridge 모델을 학습하고 RMSE를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. Lasso 특성 선택 ⭐⭐

**문제**: alpha=0.1인 Lasso 모델을 학습하고, 0이 아닌 계수의 개수를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. Random Forest 학습 ⭐⭐

**문제**: n_estimators=50, max_depth=10인 Random Forest를 학습하고 성능을 평가하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 특성 중요도 추출 ⭐⭐

**문제**: Random Forest의 특성 중요도를 DataFrame으로 만들고 상위 3개를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q6. XGBoost 학습 ⭐⭐⭐

**문제**: learning_rate=0.05, max_depth=8인 XGBoost 모델을 학습하고 성능을 평가하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. LightGBM 학습 ⭐⭐⭐

**문제**: n_estimators=200, num_leaves=31인 LightGBM 모델을 학습하고 R2를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. GridSearchCV 실습 ⭐⭐⭐⭐

**문제**: Ridge의 alpha를 [0.01, 0.1, 1, 10, 100] 중에서 GridSearchCV로 최적화하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. RandomizedSearchCV 실습 ⭐⭐⭐⭐

**문제**: Random Forest의 하이퍼파라미터를 RandomizedSearchCV로 최적화하세요.
- n_estimators: 50~200
- max_depth: 5~20
- n_iter=10

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 모델 비교 종합 ⭐⭐⭐⭐⭐

**문제**: 3개 이상의 모델을 학습하고 RMSE와 R2를 비교하는 DataFrame을 만든 후, Plotly로 시각화하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q11. CatBoost 기본 학습 ⭐⭐⭐

**문제**: CatBoostRegressor를 사용하여 회귀 모델을 학습하세요.
- n_estimators=100, max_depth=6, learning_rate=0.1
- R2 스코어와 RMSE를 출력하세요.

```python
from catboost import CatBoostRegressor
```

In [ ]:
# 여기에 코드를 작성하세요


### Q12. 부스팅 3대장 비교 ⭐⭐⭐⭐

**문제**: XGBoost, LightGBM, CatBoost 세 가지 모델을 모두 학습하고, 성능(RMSE, R2)을 비교하는 DataFrame을 만드세요.
- 동일한 하이퍼파라미터 사용: n_estimators=100, max_depth=6, learning_rate=0.1
- 어떤 모델이 가장 좋은 성능을 보이나요?

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 선형 모델 핵심 요약

| 모델 | 특징 | 언제 사용? |
|------|------|------------|
| Linear Regression | 가장 기본, 해석 용이 | 기준 모델, 선형 관계 |
| Ridge (L2) | 계수 축소 | 다중공선성, 과적합 방지 |
| Lasso (L1) | 특성 선택 | 핵심 변수 추출 |
| ElasticNet | L1+L2 조합 | Ridge와 Lasso 장점 결합 |

### Part 2: 비선형 모델 & 튜닝 요약

| 모델 | 특징 | 핵심 파라미터 |
|------|------|---------------|
| Decision Tree | 비선형 학습 | max_depth, min_samples_split |
| Random Forest | 앙상블, 안정적 | n_estimators, max_depth |
| XGBoost | 고성능, Kaggle 인기 | learning_rate, max_depth |
| LightGBM | 빠른 속도, 대용량 | num_leaves, learning_rate |
| CatBoost | 범주형 자동 처리 | cat_features, learning_rate |

### 💡 부스팅 3대장 선택 가이드

| 상황 | 추천 모델 |
|------|----------|
| 범주형 특성이 많을 때 | **CatBoost** |
| 대용량 데이터 | **LightGBM** |
| 안정적인 성능 필요 | **XGBoost** |
| 빠른 시작/일반적인 경우 | XGBoost 또는 LightGBM |

### 💡 실무 팁

1. **기준 모델부터**: Linear Regression으로 시작하여 baseline 설정
2. **스케일링 주의**: 선형 모델은 스케일링 필수, 트리 모델은 불필요
3. **과적합 방지**: 정규화(Ridge/Lasso) 또는 트리 깊이 제한
4. **교차 검증**: GridSearchCV/RandomizedSearchCV 필수
5. **앙상블 효과**: 단일 모델보다 Random Forest, Boosting이 대체로 우수
6. **CatBoost 활용**: 범주형 특성 많으면 인코딩 없이 바로 사용